In [25]:
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='browser'
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import scipy as sp
import scipy.cluster
import math


In [ ]:
# N = len(density_data)
# d = 3
# root = N**(1/d)
# H = 1 + math.log10(root/10) / math.log10(2)
# print(H)

In [27]:
density_data = pd.read_csv("../../Curated_data/final_dataset_3_no_0.csv", low_memory=False)
density_data

,Name,X,Y,Z,Density_gcm3,RQD_Pct,Cr_ppm,CP_Total,PO_Total,PY_Total,...,phi_6393,phi_6394,phi_6395,phi_6396,phi_6397,phi_6398,phi_6399,phi_6400,phi_6401,phi_6402
0,KV-NME001,0.437814,0.509816,0.461455,0.400922,0.8800,0.127305,0.250,0.066667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,KV-NME001,0.438061,0.509789,0.460591,0.410138,0.8800,0.160479,0.250,0.066667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,KV-NME001,0.448174,0.508800,0.426068,0.442396,0.9900,0.128743,0.250,0.066667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,KV-NME001,0.448431,0.508777,0.425204,0.442396,0.9900,0.141317,0.375,0.133333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,KV-NME001,0.448683,0.508755,0.424340,0.442396,0.9900,0.153293,0.500,0.200000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2613,KV365,0.629186,0.001755,0.432766,0.543779,0.9235,0.078443,0.050,0.200000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2614,KV365,0.629096,0.001314,0.431988,0.539171,0.9235,0.074850,0.050,0.200000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2615,KV365,0.629011,0.000877,0.431205,0.543779,0.8584,0.076647,0.050,0.200000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2616,KV365,0.628921,0.000436,0.430427,0.525346,0.8584,0.077246,0.050,0.200000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
deposit_data = density_data.iloc[:, :10]
X = deposit_data['X']
Y = deposit_data['Y']
Z = deposit_data['Z']

In [ ]:

lon = deposit_data.values[:, 1]
lat = deposit_data.values[:, 2]
az = deposit_data.values[:, 3]

# Subtract the minimum values for each array
lon = lon - np.min(lon)
lat = lat - np.min(lat)
az = az - np.min(az)

# Calculate the overall minimum and maximum values from the entire dataset
min_value = np.min([np.min(lon), np.min(lat), np.min(az)])
max_value = np.max([np.max(lon), np.max(lat), np.max(az)])

# Scale the arrays to (0, 1) using the calculated minimum and maximum values
normalized_lon = (lon - min_value) / (max_value - min_value)
normalized_lat = (lat - min_value) / (max_value - min_value)
normalized_az = (az - min_value) / (max_value - min_value)

N = lon.shape[0]

# Display the normalized arrays
print("Normalized Lon:", normalized_lon)
print("Normalized Lat:", normalized_lat)
print("Normalized Az:", normalized_az)


In [ ]:
num_basis = [10**3,19**3,37**3]
knots_1dx = [np.linspace(0,1,int(i**(1/3))+1) for i in num_basis]
knots_1dy = [np.linspace(0,1,int(i**(1/3))+1) for i in num_basis]
knots_1dz = [np.linspace(0,1,int(i**(1/3))+1) for i in num_basis]


basis_size = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/(num_basis[res]**(1/3))*2.5
    knots_x, knots_y, knots_z = np.meshgrid(knots_1dx[res],knots_1dy[res], knots_1dz[res])
    knots = np.column_stack((knots_x.flatten(),knots_y.flatten(),knots_z.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(np.vstack((normalized_lon,normalized_lat, normalized_az)).astype(float).T-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + basis_size] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + basis_size] = 0
    basis_size = basis_size + num_basis[res]

In [ ]:
num_basis = [10**3]
knots_1dx = [np.linspace(0,1,int(i**(1/3))+1) for i in num_basis]
knots_1dy = [np.linspace(0,1,int(i**(1/3))+1) for i in num_basis]
knots_1dz = [np.linspace(0,1,int(i**(1/3))+1) for i in num_basis]



basis_size = 0
phi = np.zeros((N, sum(num_basis)))
#For each level of resolution
for res in range(len(num_basis)):
    theta = 1/(num_basis[res]**(1/3))*2.5
    knots_x, knots_y, knots_z = np.meshgrid(knots_1dx[res],knots_1dy[res], knots_1dz[res])
    knots = np.column_stack((knots_x.flatten(),knots_y.flatten(),knots_z.flatten()))
    #For each node in the grid
    for i in range(num_basis[res]):
        d = np.linalg.norm(np.vstack((normalized_lon,normalized_lat, normalized_az)).astype(float).T-knots[i,:],axis=1)/theta
        # For each distance of our data to the node i, calculate Wendland kernel
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + basis_size] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + basis_size] = 0
    basis_size = basis_size + num_basis[res]

In [ ]:
## Romove the all-zero columns
idx_zero = np.array([], dtype=int)
for i in range(phi.shape[1]):
    if sum(phi[:,i]!=0)==0:
        idx_zero = np.append(idx_zero,int(i))

phi_reduce = np.delete(phi,idx_zero,1)
print(phi.shape)
print(phi_reduce.shape)

In [ ]:
phi_reduce = phi

In [ ]:

len_phi = np.shape(phi_reduce)[1]

# Create a DataFrame
df = pd.DataFrame(phi_reduce, columns=[f'phi_{i}' for i in range(len_phi)])

# Display the DataFrame
print(df)


In [ ]:
df_reset = df.reset_index(drop=True)
deposit_data_reset = deposit_data.reset_index(drop=True)

# Concatenate along columns
deposit_data = pd.concat([deposit_data_reset, df], axis=1)

# Display the resulting DataFrame
deposit_data

In [ ]:
# Assuming deposit_data is your DataFrame
# Extract the names of the first 98 columns
phi_columns = deposit_data.columns[10:].tolist()

# Display the list of column names
print(phi_columns[:10])


In [ ]:
total_columns = ['CP_Total','PO_Total', 'PY_Total']

#all covariates
covariates = total_columns[:3] + ['RQD_Pct', 'Cr_ppm'] 

In [ ]:
deposit_data = deposit_data.dropna(subset=['Density_gcm3'] + covariates + phi_columns)
deposit_data

In [ ]:
deposit_data.to_csv('final_dataset_1_delimited.csv', index=False)


## BEST DEEPKRIGING NORMAL

In [ ]:
p = len(phi_columns) + len(covariates)

x = deposit_data[phi_columns + covariates].values #[:,np.newaxis] makes the second dimension explicit
y = deposit_data[[variable]].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
x = x.reshape(len(deposit_data),p)
x = x[:,[i for i in range(p)]]

x = torch.tensor(x)
y = torch.tensor(y)
x = x.to(torch.float32)
y = y.to(torch.float32)


# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

torch.manual_seed(42)
np.random.seed(42)

# Assuming deposit_data, covariates, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_rmse_list = []
test_mae_list = []
test_r2_list = []



# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train = train_data[phi_columns + covariates].values
    y_train = train_data['Density_gcm3'].values

    x_test = test_data[phi_columns + covariates].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network
    model = nn.Sequential(
        nn.Linear(in_features=p, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5) ,
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=1))


    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))



    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")

In [ ]:
p = len(phi_columns) + len(covariates)

x = deposit_data[phi_columns + covariates].values #[:,np.newaxis] makes the second dimension explicit
y = deposit_data[[variable]].values[:,np.newaxis] #[:,np.newaxis] makes the second dimension explicit
x = x.reshape(len(deposit_data),p)
x = x[:,[i for i in range(p)]]

x = torch.tensor(x)
y = torch.tensor(y)
x = x.to(torch.float32)
y = y.to(torch.float32)


# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

torch.manual_seed(42)
np.random.seed(42)

# Assuming deposit_data, covariates, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_rmse_list = []
test_mae_list = []
test_r2_list = []



# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train = train_data[phi_columns + covariates].values
    y_train = train_data['Density_gcm3'].values

    x_test = test_data[phi_columns + covariates].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network
    model = nn.Sequential(
        nn.Linear(in_features=p, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5) ,
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=1))


    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))



    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")

In [ ]:
deposit_data[phi_columns + covariates]

## MODIFIED

In [ ]:
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# torch.manual_seed(42)
# np.random.seed(42)

# # Function to print evaluation metrics
# def print_metrics(actual, predicted, set_name):
#     mse = mean_squared_error(actual, predicted)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(actual, predicted)
#     r2 = r2_score(actual, predicted)

#     print(f"Metrics for {set_name} set:")
#     print(f"  MSE: {mse:.4f}")
#     print(f"  RMSE: {rmse:.4f}")
#     print(f"  MAE: {mae:.4f}")
#     print(f"  R^2: {r2:.4f}\n")

# # Assuming deposit_data, covariates, and other necessary variables are defined

# # Create an array to store metrics for each fold
# test_mse_list = []
# test_mae_list = []
# test_r2_list = []

# # Define the number of folds for cross-validation
# num_folds = 10
# kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# # Define a neural network with separate branches for phi_columns and covariates
# class GroupedNet(nn.Module):
#     def __init__(self, input_size_phi, input_size_covariates, output_size):
#         super(GroupedNet, self).__init__()
#         self.phi_branch = nn.Sequential(
#             nn.Linear(in_features=input_size_phi, out_features=100),
#             nn.Linear(in_features=100, out_features=1)
     
#         )
#         self.covariates_branch = nn.Sequential(
#             nn.Linear(in_features=input_size_covariates, out_features=100),
#             nn.Linear(in_features=100, out_features=1)       
#           )
        
#         self.combine_layer = nn.Sequential(
#             nn.Linear(2, 100), 
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.BatchNorm1d(100),
#             nn.Linear(in_features=100, out_features=100),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(in_features=100, out_features=100),
#             nn.ReLU(),
#             nn.BatchNorm1d(100),
#             nn.Linear(in_features=100, out_features=1)
#         )
                                           

#     def forward(self, input_phi, input_covariates):
#         phi_output = self.phi_branch(input_phi)
#         covariates_output = self.covariates_branch(input_covariates)
#         x = torch.cat((phi_output, covariates_output), dim=1)
#         output = self.combine_layer(x)
#         return output

# # Perform k-fold cross-validation
# for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
#     train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

#     x_train_phi = train_data[phi_columns].values
#     x_train_covariates = train_data[covariates].values
#     y_train = train_data['Density_gcm3'].values

#     x_test_phi = test_data[phi_columns].values
#     x_test_covariates = test_data[covariates].values
#     y_test = test_data['Density_gcm3'].values

#     # Define your neural network
#     model = GroupedNet(input_size_phi=len(phi_columns), input_size_covariates=len(covariates), output_size=1)

#     mse_loss = nn.MSELoss()
#     optimizer = optim.Adam(model.parameters(), lr=0.001)

#     train_losses = []  # To store training losses during training
#     test_losses = []   # To store test losses during training

#     # Training loop
#     for step in range(601):
#         pre = model(torch.tensor(x_train_phi, dtype=torch.float32), torch.tensor(x_train_covariates, dtype=torch.float32))
#         mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
#         cost = mse

#         optimizer.zero_grad()
#         cost.backward()
#         optimizer.step()

#         pre_test = model(torch.tensor(x_test_phi, dtype=torch.float32), torch.tensor(x_test_covariates, dtype=torch.float32))
#         mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
#         test_losses.append(mse_test.item())

#     # Store metrics for this fold
#     test_predictions_fold = model(torch.tensor(x_test_phi, dtype=torch.float32), torch.tensor(x_test_covariates, dtype=torch.float32)).detach().numpy().flatten()
#     test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
#     test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
#     test_r2_list.append(r2_score(y_test, test_predictions_fold))

#     # Print metrics for the current fold
#     print(f"\nMetrics for Fold {fold + 1}:")
#     print_metrics(y_test, test_predictions_fold, "Test")

# # Print average
# # Print average metrics across folds
# print("\nAverage Metrics Across Folds:")
# print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
# print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
# print(f"  Average R2: {np.mean(test_r2_list):.4f}")
# print(f"  STD MSE: {np.std(test_mse_list):.4f}")
# print(f"  STD MAE: {np.std(test_mae_list):.4f}")
# print(f"  STD R2: {np.std(test_r2_list):.4f}")

## BEST MODIFIED (Deepkriging)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

torch.manual_seed(42)
np.random.seed(42)


# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")

# Assuming deposit_data, covariates, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_mae_list = []
test_r2_list = []

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define a neural network with separate branches for phi_columns and covariates
class GroupedNet(nn.Module):
    def __init__(self, input_size_phi, input_size_covariates, output_size):
        super(GroupedNet, self).__init__()
        self.phi_branch = nn.Sequential(
            nn.Linear(in_features=input_size_phi, out_features=1)
     
        )
        self.covariates_branch = nn.Sequential(
            nn.Linear(in_features=input_size_covariates, out_features=1)      
          )
        
        self.combine_layer = nn.Sequential(
            nn.Linear(2, 100), 
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=1)
        )
                                           

    def forward(self, input_phi, input_covariates):
        phi_output = self.phi_branch(input_phi)
        covariates_output = self.covariates_branch(input_covariates)
        x = torch.cat((phi_output, covariates_output), dim=1)
        output = self.combine_layer(x)
        return output

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train_phi = train_data[phi_columns].values
    x_train_covariates = train_data[covariates].values
    y_train = train_data['Density_gcm3'].values

    x_test_phi = test_data[phi_columns].values
    x_test_covariates = test_data[covariates].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network
    model = GroupedNet(input_size_phi=len(phi_columns), input_size_covariates=len(covariates), output_size=1)

    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train_phi, dtype=torch.float32), torch.tensor(x_train_covariates, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test_phi, dtype=torch.float32), torch.tensor(x_test_covariates, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test_phi, dtype=torch.float32), torch.tensor(x_test_covariates, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))

    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average
# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")

## NN covariates

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

torch.manual_seed(42)
np.random.seed(42)

phi_columns = ['X','Y','Z']
# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")

# Assuming deposit_data, covariates, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_mae_list = []
test_r2_list = []

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define a neural network with separate branches for phi_columns and covariates
class GroupedNet(nn.Module):
    def __init__(self, input_size_phi, input_size_covariates, output_size):
        super(GroupedNet, self).__init__()
        self.phi_branch = nn.Sequential(
            nn.Linear(in_features=input_size_phi, out_features=1)
     
        )
        self.covariates_branch = nn.Sequential(
            nn.Linear(in_features=input_size_covariates, out_features=1)      
          )
        
        self.combine_layer = nn.Sequential(
            nn.Linear(2, 100), 
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=1)
        )
                                           

    def forward(self, input_phi, input_covariates):
        phi_output = self.phi_branch(input_phi)
        covariates_output = self.covariates_branch(input_covariates)
        x = torch.cat((phi_output, covariates_output), dim=1)
        output = self.combine_layer(x)
        return output

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train_phi = train_data[phi_columns].values
    x_train_covariates = train_data[covariates].values
    y_train = train_data['Density_gcm3'].values

    x_test_phi = test_data[phi_columns].values
    x_test_covariates = test_data[covariates].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network
    model = GroupedNet(input_size_phi=len(phi_columns), input_size_covariates=len(covariates), output_size=1)

    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train_phi, dtype=torch.float32), torch.tensor(x_train_covariates, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test_phi, dtype=torch.float32), torch.tensor(x_test_covariates, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test_phi, dtype=torch.float32), torch.tensor(x_test_covariates, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))

    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average
# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")

## Deepkriging no covariates

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

torch.manual_seed(42)
np.random.seed(42)

# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")

# Assuming deposit_data, phi_columns, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_mae_list = []
test_r2_list = []

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define a neural network without covariates
class GroupedNet(nn.Module):
    def __init__(self, input_size_phi, output_size):
        super(GroupedNet, self).__init__()
        self.phi_branch = nn.Sequential(
            nn.Linear(in_features=input_size_phi, out_features=1)
        )
        
        self.combine_layer = nn.Sequential(
            nn.Linear(1, 100), 
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=1)
        )

    def forward(self, input_phi):
        phi_output = self.phi_branch(input_phi)
        x = phi_output
        output = self.combine_layer(x)
        return output

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train_phi = train_data[phi_columns].values
    y_train = train_data['Density_gcm3'].values

    x_test_phi = test_data[phi_columns].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network without covariates
    model = GroupedNet(input_size_phi=len(phi_columns), output_size=1)

    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train_phi, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test_phi, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test_phi, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))

    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")


In [ ]:
deposit_data[['X','Y','Z']]

## NN no covariates

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

phi_columns = ['X','Y','Z']

torch.manual_seed(42)
np.random.seed(42)

# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")

# Assuming deposit_data, phi_columns, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_mae_list = []
test_r2_list = []

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define a neural network without covariates
class GroupedNet(nn.Module):
    def __init__(self, input_size_phi, output_size):
        super(GroupedNet, self).__init__()
        self.phi_branch = nn.Sequential(
            nn.Linear(in_features=input_size_phi, out_features=1)
        )
        
        self.combine_layer = nn.Sequential(
            nn.Linear(1, 100), 
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=1)
        )

    def forward(self, input_phi):
        phi_output = self.phi_branch(input_phi)
        x = phi_output
        output = self.combine_layer(x)
        return output

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train_phi = train_data[phi_columns].values
    y_train = train_data['Density_gcm3'].values

    x_test_phi = test_data[phi_columns].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network without covariates
    model = GroupedNet(input_size_phi=len(phi_columns), output_size=1)

    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train_phi, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test_phi, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test_phi, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))

    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")
